## **Logisitc Regression with L1 penalty prediction**

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import glob
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 100000

import gc
from myUtils import *
from feature_generator import feature_v1
import xgboost as xgb
from tqdm import tqdm
import shutil
import copy

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

import pickle

import warnings
warnings.simplefilter('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def label_encode(df, cols):
    for col in cols:
        le = LabelEncoder()
        tmp = df[col].fillna("NaN")
        df[col] = pd.Series(le.fit_transform(tmp), index=tmp.index)

    return df


def check_columns(necessary_cols,cols):
    
    cols = set(cols) # make set
    
    lack_cols = [c for c in necessary_cols if c not in cols]
    
    print("-- column check completed --")
    if len(lack_cols) == 0:
        print("  columns are satisfied")
        return True
    else:
        print("  !!columns are lacked!!")
        print("   lacked columns:",lack_cols)
        return False


class FeaturesMaker_v1(object):

    def __init__(self,target_col):
        self.name = "featuresV1"
        self.feature_exp = "simple features which "

        self.target_col = target_col
        self.necessary_col =  ["sig_id",'cp_type',"cp_time","cp_dose","data_part"] + [target_col]

    def make_feature(self,df):

        # check existstance of necessary columns
        if check_columns(self.necessary_col,df.columns):

            # label encoding
            cols = ['cp_type',"cp_time","cp_dose"]
            df = label_encode(df, cols=cols)


            # split train and test
            df = df.set_index(["sig_id"],drop=True)

            features = [c for c in df.columns if "g-" in c]
            features = features + [c for c in df.columns if "c-" in c]
            features = features + ['cp_type',"cp_time","cp_dose"]

            print("-- ",self.name," --")
            print("dim:",len(features))
            print("N:",len(df))
            print("-----------------")

            return {sub[0]:(sub[1][features],sub[1][self.target_col]) for sub in df.groupby(by="data_part")}

        else:
            return False

In [6]:
train_targets = pd.read_csv(os.path.join("..","input","lish-moa","train_targets_scored.csv"))
train_features = pd.read_csv(os.path.join("..","input","lish-moa","train_features.csv"))
test_features =  pd.read_csv(os.path.join("..","input","lish-moa","test_features.csv"))


sample_submission = pd.read_csv(os.path.join("rawdata","sample_submission.csv"),index_col=0)
submission = pd.DataFrame(index=sample_submission.index)

for target in train_targets.columns[1:]:
    feature_maker = FeaturesMaker_v1(target_col=target)

    train_data = copy.copy(train_features)
    train_data = pd.merge(train_data,train_targets[["sig_id",target]],on="sig_id",how="right")
    train_data["data_part"] = "train"

    test_data = copy.copy(test_features)
    test_data[target] = np.nan
    test_data["data_part"] = "test"

    data = pd.concat([train_data,test_data])
    data = feature_maker.make_feature(data)
    
    dir = os.path.join("..","input",feature_maker.name+"_LogisticRegL1")
    model_path = os.path.join(dir,target+".mdl")      
    
    # training XGBoost
    with open(model_path,"rb") as f:
        model = pickle.load(f)
    
    pred_mask = data["test"][1].isna()
    data["test"][1].loc[pred_mask] = model.predict_proba(data["test"][0])[:,1]
    
    submission = submission.join(pd.DataFrame(data["test"][1]))

-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --
  columns are satisfied
--  featuresV1  --
dim: 875
N: 27796
-----------------
-- column check completed --

KeyboardInterrupt: 